<a href="https://colab.research.google.com/github/gabrielfernandorey/ITBA-NLP/blob/main/ITBA_nlp01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico NLP - Detección de Tópicos y clasificación
- ITBA 2024
- Alumno: Gabriel Rey
---

### Resumen del problema

- Calcular los tópicos de portales de noticias que se reciben 
- Frecuencia del cálculo de tópicos: diaria
- Colección de noticias: diariamente, en lotes o de a un texto.
- Identificar tópicos, entidades, keywords y análisis de sentimiento.

### Datos
- Se reciben las noticias con formato: Titulo, Texto, Fecha, Entidades, Keywords

### Tareas
- Modelo de detección de tópicos diario utilizando embeddings
- Definir un criterio de agrupación de tópicos aplicado al mismo día y entre distintos días (merging)
- Almacenar los embeddings de tópicos en una base de datos vectorial
- Modelo de datos: 
    - Id del tópico
    - Nombre del tópico
    - Keywords
    - Embbeding
    - Fecha de creación
    - Fecha de entrenamiento inicial
    - Fecha de entrenamiento actualizada
    - Umbral de detección
    - Documento mas cercano
---
Tareas en esta notebook:
- Inicializar la base de datos vectorial
- Ingestar data
- NER: Encontrar las entidades de cada documento
- Limpiar data
- Modelo: Armado del modelo BERTopic
- Entrenamiento
- Almacenamiento en base de datos vectorial


In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re
import json
from datetime import datetime
from dotenv import load_dotenv
from tqdm import tqdm
from collections import Counter

import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from NLP_tools import clean_all
from core.functions import *

# levantar la base antes de ejecutar
from opensearch_data_model import os_client

### Inicializamos la base vectorial

In [2]:
init_opensearch()

2024-07-01 19:51:27.539 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


El índice Topic ya existe. Saltando inicialización de base de datos.
El índice News ya existe. Saltando inicialización de base de datos.


### Path

In [3]:
load_dotenv()
PATH_REMOTO='/content/ITBA-NLP/data/'
PATH=os.environ.get('PATH_LOCAL', PATH_REMOTO)
PATH

'C:/Users/gabri/OneDrive/Machine Learning/Github/ITBA-TP/data/'

### Data

In [4]:
# Read the parquet file

df_params = {'0_1000':'0_1000_data.parquet',
             '1000_2000':'1000_2000_data.parquet',
             '2000_3000':'2000_3000_data.parquet',
             'df_joined':'df_joined_2024-04-01 00_00_00.parquet'
            }

chunk = os.environ.get('CHUNK')

df_parquet = pd.read_parquet(PATH+df_params[chunk])
df_parquet.head(1)


,Asset Name,Author Id,Author Name,Keyword Id,Keyword Name,Entity Id,Entity Name,Media Group Id,Media Group Name,Impact,...,in__text,out__entities,out__potential_entities,predicted_at_entities,out__keywords_sorted,predicted_at_keywords,start_time_utc,start_time_local,truncated_text,title_and_text
Asset Id,,,,,,,,,,,,,,,,,,,,,
105628101,Elecciones en Venezuela: María Corina Machado ...,36192,Infobae,1932002 | 417739 | 1687638 | 36187 | 7476 | 50...,"[falsas conspiraciones armadas, sustituta, det...",219925 | 210613 | 219770 | 36424 | 1129437,"[Nicolás Maduro, Jorge Rodríguez, Marcelo Ebra...",0,,7406333,...,Fotografía de archivo de la líder antichavista...,"[Nicolás Maduro, Marcelo Ebrard, Jorge Rodrígu...","[Jorge Rodríguez, Nicolás Maduro, Rayner Peña ...",2024-04-02 08:11:57.825777,"[elecciones presidenciales, candidatura presid...",2024-04-02 08:17:44.372891+00:00,2024-04-02,2024-04-01 21:00:00,Fotografía de archivo de la líder antichavista...,Elecciones en Venezuela: María Corina Machado ...


In [ ]:
# Codigo para fraccionar el dataset (pruebas)
#df_parquet[:1000].to_parquet(PATH+'0_1000_data.parquet', engine='pyarrow')

#df_1000_2000 = df_parquet[1000:2000]

#df_1000_2000['start_time_local'] = '2024-04-03 00:00:00'
#df_1000_2000.to_parquet(PATH+'1000_2000_data.parquet', engine='pyarrow')

#df_2000 = df_parquet[2000:]

#df_2000['start_time_local'] = '2024-04-05 00:00:00'
#df_2000.to_parquet(PATH+'2000_3000_data.parquet', engine='pyarrow')


In [5]:
data = list(df_parquet['in__text'])

# Cantidad total de documentos
len(data)

1000

### StopWords

In [6]:
# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SPECIAL = list(pd.read_csv(PATH+'spanish_stop_words_spec.csv' )['stopwords'].values)

In [117]:
""" import csv
# Guardar la lista en un archivo CSV
with open(PATH+"spanish_stop_words_spec.csv", mode='w', newline='', encoding='utf-8') as archivo:
    escritor = csv.writer(archivo)
    escritor.writerow(['stopwords'])
    for stopword in SPANISH_STOPWORDS_SPECIAL:
        escritor.writerow([stopword]) """

### NER - Named Entity Recognition
Obtener entidades de las noticias 

In [7]:
# Cargar el modelo de spaCy para español
spa = spacy.load("es_core_news_lg")

In [ ]:
""" # Cargar o saltar carga y procesar celda inferior
with open(PATH+f'modelos/entities{chunk}.json', 'r') as json_file:
    entities = json.load(json_file)

with open(PATH+f'modelos/entities_spa{chunk}.json', 'r') as json_file:
    entities_spa = json.load(json_file)

with open(PATH+f'modelos/keywords_spa{chunk}.json', 'r') as json_file:
    keywords_spa = json.load(json_file) """

In [8]:
# Detectar entidades y keywords para todos los documentos usando spaCy
entities_spa = []
keywords_spa = []
for doc in tqdm(data):
    extract = spa(doc)
    entities_spa.append([(ent.text, ent.label_) for ent in extract.ents])
    keywords_spa.append([(ext.text, ext.pos_) for ext in extract])    

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [01:36<00:00, 10.39it/s]


In [9]:
# Procesamiento de entidades encontradas
entities = []
original_entities = []
word_count = {}
for item in tqdm(entities_spa):
    for ent in item:
        if ent[1] == 'PER' or ent[1] == 'ORG' or ent[1] == 'LOC':
            words = str(ent[0]).lower()
            words = clean_all([words], accents=False)[0]
            words = " ".join(words.split())
            if len(words) > 2 and len(words.split()) <= 3:   # valida mas de una letra cada palabra & maximo 3 palabras por token
                add = True
                
                for token in words.split():
                    if token.isalpha():
                        if token in SPANISH_STOPWORDS or token in SPANISH_STOPWORDS_SPECIAL:
                            add = False
                        
                    elif token.isnumeric():
                        if len(token) > 5:
                            add = False
                    else:
                        if token in SPANISH_STOPWORDS_SPECIAL:
                            add = False

                    if add:
                        if words not in word_count:
                            word_count[words] = {'count': 0, 'original': ent[0]}
                        else:
                            word_count[words]['count'] += 1      

    # Ordenar el diccionario por el valor del conteo en orden descendente
    sorted_word_count = dict(sorted(word_count.items(), key=lambda item: item[1]['count'], reverse=True))  

    word_count = {}

    pre_original_entities = [value['original'] for _, value in sorted_word_count.items()]
                             
    # Crear la lista de entidades procesadas por noticia (entrenamiento)
    pre_entities = [key for key, _ in sorted_word_count.items()] # if _['count'] > 1]

    # Obtener las últimas palabras de las entidades con más de una palabra
    ultimas_palabras = [ent.split()[-1] for ent in pre_entities if len(ent.split()) > 1]

    # Filtrar si las últimas palabras coinciden con alguna unica palabra
    filtro_ulp = [ent for ent in pre_entities if not (len(ent.split()) == 1 and ent in ultimas_palabras)]

    # Obtener las palabras únicas
    unicas_palabras = [ ent for ent in filtro_ulp if len(ent.split()) == 1]

    # Filtrar si las palabras únicas coinciden con las una entidad con más de una palabra
    filtro_unp = [ ent for ent in filtro_ulp if not ent in unicas_palabras ]

    umbral=10
    # entidades para entrenar
    entities.append( filtro_unp[:umbral] )
    original_entities.append([pre for pre in pre_original_entities if pre.lower() in filtro_unp[:umbral]])

    

100%|██████████| 1000/1000 [00:00<00:00, 1132.50it/s]


In [16]:
# Encontrar la posicion en el df segun su indice
df_parquet.index.get_loc(105640350)

211

In [9]:
df_entities = pd.DataFrame(entities)
df_entities

,0,1,2,3,4,5,6,7,8,9
0,maría corina machado,nicolás maduro,corina yoris,gerardo blyde pérez,consejo nacional electoral,edmundo gonzález urrutia,plataforma unitaria democrática,jonas gahr,embajada argentina,vente venezuela
1,None,None,None,None,None,None,None,None,None,None
2,ptsd tept,None,None,None,None,None,None,None,None,None
3,marcos brindicci,reino unido,erica sandberg,courtney alev,teri williams,oneunited bank,None,None,None,None
4,diego maradona,guillermo coppola,enrique pinti,juanito belmonte,calle libertad,salsa criolla,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
995,río negro,alberto weretilneck,gustavo melella,rolando figueroa,puerto madryn,congreso extraordinario,tierra del fuego,ate y upcn,None,None
996,javier milei,gustavo petro,gobierno colombiano,camilo romero,gobierno argentino,diana mondino,república argentina,cancillerías de colombia,república de colombia,None
997,luis caputo,emmanuel álvarez agis,marina dal poggetto,None,None,None,None,None,None,None
998,javier milei,marcelo hacklander,None,None,None,None,None,None,None,None


In [11]:
len(entities)

1000

In [ ]:
# Grabar
with open(PATH+f'modelos/entities{chunk}.json', 'w') as file:
    json.dump(entities, file)

# Grabar
with open(PATH+f'modelos/entities_spa{chunk}.json', 'w') as file:
    json.dump(entities_spa, file)

# Grabar
with open(PATH+f'modelos/keywords_spa{chunk}.json', 'w') as file:
    json.dump(keywords_spa, file)

## Keywords
Obtener palabras clave de las noticias

In [19]:
# Procesamiento de keywords encontradas como 'MISC'
keywords = []
original_keywords = []
word_count = {}
for i, item in tqdm(enumerate(entities_spa)):
    for ent in item:
        if ent[1] == 'MISC':
            words = str(ent[0]).lower()
            words = clean_all([words], accents=False)[0]
            #if len(entities[i]) < 5: # Si se encontraron menos de 5 entidades, obtenemos keywords.
            if len(words) > 2 and len(words.split()) < 2:
                add = True
                for token in words.split():
                    if token.isalpha():
                        if token in SPANISH_STOPWORDS or token in SPANISH_STOPWORDS_SPECIAL:
                            add = False
                    elif token.isnumeric():
                        if len(token) > 5:
                            add = False
                    else:
                        if token in SPANISH_STOPWORDS_SPECIAL:
                            add = False

                    if add:
                        if words not in word_count:
                            word_count[words] = {'count': 0, 'original': ent[0]}
                        else:
                            word_count[words]['count'] += 1   


    # Ordenar el diccionario por el valor del conteo en orden descendente
    
    sorted_word_count = dict(sorted(word_count.items(), key=lambda item: item[1]['count'], reverse=True))  
    word_count = {}

    # Crear la lista de entidades procesadas por noticia (para guardar en DB)
    original_keywords.append([value['original'] for _, value in sorted_word_count.items()]) # and value['count'] > 1] )
                                
    # Crear la lista de entidades procesadas por noticia (entrenamiento)
    pre_keywords = [key for key, _ in sorted_word_count.items()]   # and _['count'] > 1]
                        
    # entidades para entrenar (seleccionamos hasta 5 primeras)
    keywords.append( pre_keywords[:5] )


0it [00:00, ?it/s]

1000it [00:00, 3538.88it/s]


In [20]:
keywords[211]

['adentro']

In [21]:
df_keywords = pd.DataFrame(keywords)
df_keywords

,0,1,2,3,4
0,None,None,None,None,None
1,None,None,None,None,None
2,None,None,None,None,None
3,newsweek,salidas”,ropa”,tarjetas,tiktok
4,tumberos,sorprendidos,disputas,quédense,diez
...,...,...,...,...,...
995,None,None,None,None,None
996,None,None,None,None,None
997,inflación,iprofesional,precios,None,None
998,dnu,None,None,None,None


### Keyboards with neighboards

In [43]:
# Prueba ejemplo
doc = 211

# Obtenemos las keywords 'NOUN' mas frecuentes
nouns = []
for token in keywords_spa[doc]:
    if token[1] == 'NOUN':
        nouns.append(token[0])

count_nouns = Counter(nouns)

count_nouns.most_common()[:10]

[('fotógrafo', 9),
 ('agua', 5),
 ('playa', 4),
 ('costa', 4),
 ('olas', 4),
 ('mar', 4),
 ('años', 4),
 ('revista', 4),
 ('día', 3),
 ('amiga', 3)]

In [42]:
# Prueba ejemplo
doc = 211

# Obtenemos las keywords 'VERB' mas frecuentes
verbs = []
for token in keywords_spa[doc]:
    if token[1] == 'VERB':
        verbs.append(token[0])

count_verbs = Counter(verbs)

count_verbs.most_common()[:10]

[('volver', 3),
 ('tiró', 3),
 ('llegar', 3),
 ('rescatarla', 2),
 ('desapareció', 2),
 ('tomar', 2),
 ('empezaron', 2),
 ('convirtió', 2),
 ('contó', 2),
 ('ocurrió', 1)]

In [24]:
# Pobar un documento ( resultados lematizados )
keywords_spa_n = []
extract = spa(data[doc])
keywords_spa_n.append([(ext.lemma_, ext.pos_) for ext in extract])
keywords_spa_n[0][:10]


[('el', 'DET'),
 ('tragedia', 'NOUN'),
 ('ocurrir', 'VERB'),
 ('el', 'DET'),
 ('primero', 'ADJ'),
 ('día', 'NOUN'),
 ('del', 'ADP'),
 ('año', 'NOUN'),
 ('en', 'ADP'),
 ('el', 'DET')]

In [25]:
# Resultados sin lematizar
extract = spa(data[doc])
tokens_and_labels = [(token.text, token.pos_) for token in extract if token.is_alpha]
tokens_and_labels[:10]

[('La', 'DET'),
 ('tragedia', 'NOUN'),
 ('ocurrió', 'VERB'),
 ('el', 'DET'),
 ('primer', 'ADJ'),
 ('día', 'NOUN'),
 ('del', 'ADP'),
 ('año', 'NOUN'),
 ('en', 'ADP'),
 ('la', 'DET')]

In [26]:
# Make a function to get all two-word combinations
def get_bigrams(word_list, number_consecutive_words=2):
    
    ngrams = []
    adj_length_of_word_list = len(word_list) - (number_consecutive_words - 1)
    
    #Loop through numbers from 0 to the (slightly adjusted) length of your word list
    for word_index in range(adj_length_of_word_list):
        
        #Index the list at each number, grabbing the word at that number index as well as N number of words after it
        ngram = word_list[word_index : word_index + number_consecutive_words]
        
        #Append this word combo to the master list "ngrams"
        ngrams.append(ngram)
        
    return ngrams

In [27]:
bigrams = get_bigrams(tokens_and_labels)
bigrams[:10]


[[('La', 'DET'), ('tragedia', 'NOUN')],
 [('tragedia', 'NOUN'), ('ocurrió', 'VERB')],
 [('ocurrió', 'VERB'), ('el', 'DET')],
 [('el', 'DET'), ('primer', 'ADJ')],
 [('primer', 'ADJ'), ('día', 'NOUN')],
 [('día', 'NOUN'), ('del', 'ADP')],
 [('del', 'ADP'), ('año', 'NOUN')],
 [('año', 'NOUN'), ('en', 'ADP')],
 [('en', 'ADP'), ('la', 'DET')],
 [('la', 'DET'), ('playa', 'NOUN')]]

In [28]:
# return the most frequent words that appear next to a particular keyword
def get_neighbor_words(keyword, bigrams, pos_label = None):
    
    neighbor_words = []
    keyword = keyword.lower()
    
    for bigram in bigrams:
        
        #Extract just the lowercased words (not the labels) for each bigram
        words = [word.lower() for word, label in bigram]        
        
        #Check to see if keyword is in the bigram
        if keyword in words:
            idx = words.index(keyword)
            for word, label in bigram:
                
                #Now focus on the neighbor word, not the keyword
                if word.lower() != keyword:
                    #If the neighbor word matches the right pos_label, append it to the master list
                    if label == pos_label or pos_label == None:
                        if idx == 0:
                            neighbor_words.append(" ".join([keyword, word.lower()]))
                        else:
                            neighbor_words.append(" ".join([word.lower(), keyword]))
                    
    return Counter(neighbor_words).most_common()

In [44]:
for word in count_nouns.most_common():
    print(get_neighbor_words(word[0], bigrams, pos_label='ADJ'))

[]
[('agua junto', 1)]
[('playa junto', 1)]
[]
[]
[]
[]
[]
[('primer día', 1), ('día siguiente', 1)]
[]
[]
[]
[]
[]
[]
[]
[('gritos desesperados', 2)]
[]
[]
[]
[]
[]
[]
[('grandísimo amigo', 1)]
[('año nuevo', 1)]
[('chacra marítima', 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('reporteros gráficos', 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('médico forense', 1)]
[('paro cardíaco', 1)]
[]
[('fuerte diferencia', 1), ('diferencia térmica', 1)]
[]
[]
[]
[]
[]
[]
[]
[]
[('gran reportero', 1), ('reportero gráfico', 1)]
[]
[]
[('reacción heroica', 1)]
[]
[]
[('condición humana', 1)]


### Funcion completa para keywords with neighboards

In [48]:
def keywords_with_neighboards(keywords_spa, POS_1='NOUN', POS_2='ADJ'):
    """
    Funcion que devuelve dos listas:
    - lista de keywords with neighboards (segun argumentos POS_1 y POS_2)
    - lista de keywords mas frecuentes (segun argumentos POS_1 y POS_2)
    """

    doc_kwn = []
    commons = []
    for keywords in keywords_spa:
    
        # Obtenemos las keywords del tipo (Universal Dependences) mas frecuentes de cada doc (spaCy format)
        words = []
        for k_spa in keywords:
            if k_spa[1] == POS_1:
                words.append(k_spa[0])

        cont_words = Counter(words)

        common = cont_words.most_common()
        commons.append( [com for com in common if com[1] > 1] )

        # Calcular un umbral de corte (en repeticiones) para los keywords obtenidos
            ## suma de todos los valores
        valores = [valor for _, valor in common]

            ## Calcular los pesos como proporcionales a los valores mismos
        pesos = np.array(valores) / np.sum(valores)

            ## Calcular el umbral ponderado, valor 2 o superior ( debe repetirse la keyword al menos una vez )
        threshold = max(2, round(np.sum(np.array(valores) * pesos),4))


        # Obtenemos los bigramas del doc        
        tokens_and_labels = [(token[0], token[1]) for token in keywords if token[0].isalpha()]

        bigrams = get_bigrams(tokens_and_labels)

        keywords_neighbor = []
        for item_common in common:
            if item_common[1] >= threshold or len(keywords_neighbor) < 6: # corte por umbral o menor a 6
                
                kwn = get_neighbor_words(item_common[0], bigrams, pos_label=POS_2)
                if kwn != []:
                    keywords_neighbor.append( kwn )

        sorted_keywords_neighbor = sorted([item for sublist in keywords_neighbor for item in sublist ], key=lambda x: x[1], reverse=True)
        
        doc_kwn.append(sorted_keywords_neighbor)

    return doc_kwn, commons

In [49]:
k_w_n, common = keywords_with_neighboards(keywords_spa)

In [57]:
# filtramos que al menos se repitan una vez
filtered_k_w_n = [ [tupla[0] for tupla in sublista if tupla[1] > 1] for sublista in k_w_n ]
filtered_k_w_n[211]


['gritos desesperados']

In [47]:
common[211]

[('fotógrafo', 9),
 ('agua', 5),
 ('playa', 4),
 ('costa', 4),
 ('olas', 4),
 ('mar', 4),
 ('años', 4),
 ('revista', 4),
 ('día', 3),
 ('amiga', 3),
 ('ayuda', 3),
 ('conductor', 3),
 ('héroe', 3),
 ('tragedia', 2),
 ('hija', 2),
 ('guardia', 2),
 ('gritos', 2),
 ('equipos', 2),
 ('arena', 2),
 ('chica', 2),
 ('cuatriciclo', 2),
 ('personas', 2),
 ('hombre', 2),
 ('amigo', 2)]

In [58]:
filtered_common = [ [tupla[0] for i, tupla in enumerate(sublista) if i < 6] for sublista in common ]


In [61]:
filtered_common[211]

['fotógrafo', 'agua', 'playa', 'costa', 'olas', 'mar']

#### BOW - Armado del vocabulario con las entidades y keywords

In [59]:
# Unificar Entities + Keywords + Keywords with neighboards
vocab = list(set().union(*entities, *keywords, *filtered_k_w_n, *common[:10]))
len(vocab)

6251

In [60]:
vocab[211]

'emmanuel soliño'

In [53]:
# Guardar vocabulario
with open(PATH+f'modelos/vocabulary{chunk}.json', 'w') as file:
    json.dump(vocab, file)

### Guardar noticias en la base

In [ ]:
# configurar  batch_size = ( ej.: 5000 ) si se supera el limite 100MB en elasticsearch por operacion
index_name = 'news'
bulk_data = []

# Unificar Keywords + Keywords with neighboards
keywords_plus = [ list(set(keywords[i]+filtered_k_w_n[i])) for i in range(len(entities)) ]

for idx, text_news in tqdm(enumerate(data)):
    doc = {
        'index': {
            '_index': index_name,
            '_id': int(df_parquet.index[idx])
        }
    }
    reg = {
        'title': str(df_parquet.iloc[idx].in__title),
        'news' : str(text_news), 
        'author': str(df_parquet.iloc[idx]['Author Name']),
        'topics': {},
        'vector': None,
        'keywords' : keywords_plus[idx],
        'entities' : original_entities[idx],
        'created_at': datetime.now().isoformat(),
        'process': False
    }
    bulk_data.append(json.dumps(doc))
    bulk_data.append(json.dumps(reg))

# Convertir la lista en un solo string separado por saltos de línea
bulk_request_body = '\n'.join(bulk_data) + '\n'

# Enviar la solicitud bulk
response = os_client.bulk(body=bulk_request_body)

if response['errors']:
    print("Errores encontrados al insertar los documentos")
else:
    print("Documentos insertados correctamente")


### Nota:
- por cada documento se van a guardar las entidades que al menos se repitan una vez (mayor frecuencia)
- se utilizarán todas las entidades guardadas de todos los documentos como vocabulario.

In [75]:
def funcion_aux(ID):
    keywords_df = df_parquet[df_parquet.index==ID]['Keyword Name'].values
    entities_df = df_parquet[df_parquet.index==ID]['Entity Name'].values
    fila = df_parquet.index.get_loc(ID)
    print(f"Keywords de dataframe: {keywords_df}")
    print(f"Entities de dataframe: {entities_df}")
    print("-"*80)
    print(f"Fila: {fila}")
    print(f"Entities calculadas: {entities[fila]}")
    print(f"Keywords calculadas: {keywords[fila]}")
    print(f"Keywords neighboards calculadas: {filtered_k_w_n[fila]}")

funcion_aux(105579385)

Keywords de dataframe: [array(['sitio oficial', 'membresía', 'suscripción', 'dólar',
        'plan deseado'], dtype=object)                       ]
Entities de dataframe: [array(['Netflix'], dtype=object)]
--------------------------------------------------------------------------------
Fila: 886
Entities calculadas: ['netflix', 'gobierno nacional', 'pais', 'ganancias', '―celular', 'nacion']
Keywords calculadas: []
Keywords neighboards calculadas: ['dólar oficial', 'sitio oficial']
